In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the pre-trained DialoGPT-medium model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Hospital-related responses
hospital_data = {
    "hours": "We are open 24/7, feel free to visit any time.",
    "location": "Our hospital is located at 123 Health St, Wellness City.",
    "emergency": "In case of an emergency, please call 911 immediately or visit our emergency department.",
    "doctor": "We have a team of expert doctors in various specialties. Would you like to know about a specific doctor?",
    "appointment": "To schedule an appointment, please call our front desk at 555-1234 or visit our website.",
    "facilities": "Our hospital is equipped with state-of-the-art facilities including a 24/7 emergency room, a maternity ward, and a well-equipped ICU."
}

# Function to generate a response from the model
def generate_response(user_input):
    # Check if the user input contains any keywords related to hospital info
    user_input_lower = user_input.lower()

    # Look for keywords in the user input and respond with relevant hospital info
    for keyword, response in hospital_data.items():
        if keyword in user_input_lower:
            return response

    # If no keyword is found, proceed with the model-based response
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # If a GPU is available, move the model to the GPU
    if torch.cuda.is_available():
        new_user_input_ids = new_user_input_ids.to("cuda")
        model.to("cuda")

    # Generate a response from the model
    chat_history_ids = model.generate(new_user_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id, no_repeat_ngram_size=3)

    # Decode the response
    bot_response = tokenizer.decode(chat_history_ids[:, new_user_input_ids.shape[-1]:][0], skip_special_tokens=True)

    return bot_response

def chatbot():
    print("Chatbot: Hello! How can I assist you today? Type 'quit' to exit.")

    # Loop for interacting with the user
    while True:
        user_input = input("You: ")

        if user_input.lower() == "quit":
            print("Chatbot: Goodbye!")
            break

        response = generate_response(user_input)
        print(f"Chatbot: {response}")

# Start the chatbot
if __name__ == "__main__":
    chatbot()


Chatbot: Hello! How can I assist you today? Type 'quit' to exit.
You: doctors?
Chatbot: We have a team of expert doctors in various specialties. Would you like to know about a specific doctor?
You: quit
Chatbot: Goodbye!
